Let's analyze the data for Toxic Release emissions on Land:

In [2]:
import pandas as pd
import fpsnippets

In [3]:
data = pd.read_csv("land-toxic-release.csv",  usecols = ['CITY_NAME','COUNTY_NAME', 'STATE_ABBR', 'ZIP_CODE', 'REPORTING_YEAR', 'CHEM_NAME', 'CARCINOGEN'])
data[700:710]

,CITY_NAME,COUNTY_NAME,STATE_ABBR,ZIP_CODE,REPORTING_YEAR,CHEM_NAME,CARCINOGEN
700,PITTSBURGH,ALLEGHENY,PA,15225,1997,Chromium,N
701,PITTSBURGH,ALLEGHENY,PA,15225,1997,Chromium,N
702,PITTSBURGH,ALLEGHENY,PA,15225,1997,Chromium,N
703,PITTSBURGH,ALLEGHENY,PA,15225,1997,Chromium,N
704,CREIGHTON,ALLEGHENY,PA,15030,1997,Ethylene glycol,N
705,CREIGHTON,ALLEGHENY,PA,15030,1997,Ethylene glycol,N
706,CREIGHTON,ALLEGHENY,PA,15030,1997,Ethylene glycol,N
707,CREIGHTON,ALLEGHENY,PA,15030,1997,Ethylene glycol,N
708,CREIGHTON,ALLEGHENY,PA,15030,1997,Ethylene glycol,N
709,CREIGHTON,ALLEGHENY,PA,15030,1997,Methanol,N


In [4]:
stateMask = data['STATE_ABBR'] == "PA"
cityMask = data['CITY_NAME'] == "PITTSBURGH"
countyMask = data['COUNTY_NAME']== "ALLEGHENY"

#let's use these query masks to get only the info for Allegheny County in Pittsburgh,PA
relevantData = data[stateMask & cityMask & countyMask]
relevantData[700:710]

,CITY_NAME,COUNTY_NAME,STATE_ABBR,ZIP_CODE,REPORTING_YEAR,CHEM_NAME,CARCINOGEN
1810,PITTSBURGH,ALLEGHENY,PA,15225,1996,Acrylic acid,N
1811,PITTSBURGH,ALLEGHENY,PA,15225,1996,Acrylic acid,N
1812,PITTSBURGH,ALLEGHENY,PA,15225,1996,Acrylic acid,N
1813,PITTSBURGH,ALLEGHENY,PA,15225,1996,Acrylic acid,N
1814,PITTSBURGH,ALLEGHENY,PA,15225,1996,Ethylbenzene,Y
1815,PITTSBURGH,ALLEGHENY,PA,15225,1996,Ethylbenzene,Y
1816,PITTSBURGH,ALLEGHENY,PA,15225,1996,Ethylbenzene,Y
1817,PITTSBURGH,ALLEGHENY,PA,15225,1996,Ethylbenzene,Y
1818,PITTSBURGH,ALLEGHENY,PA,15225,1996,Ethylbenzene,Y
1819,PITTSBURGH,ALLEGHENY,PA,15225,1996,Chromium,N


In [6]:
#Getting a list of unique zip codes in the area
zips = relevantData['ZIP_CODE'].unique()
zips.sort()

#Now, let's count the number of carniogens found in each zip code
x = 0;
for zip in zips:
    carcCounter = 0;
    mask = relevantData['ZIP_CODE'] == zip
    maskedData = relevantData[mask]
    for row in maskedData:
        carcYorN = relevantData.iloc[x,6]
        if carcYorN == "Y":
            carcCounter = carcCounter + 1
        x = x + 1
    #Let's print the zips and neighborhoods with no carcinogens found
    if carcCounter == 0:
        print(zip, ":", carcCounter, " Carcinogens found")
        print("Neighborhoods in zip", zip, ":", fpsnippets.zip_to_neighborhoods(zip), "\n")

15203 : 0  Carcinogens found
Neighborhoods in zip 15203 : ['Arlington Heights', 'Allentown', 'South Side Slopes', 'Arlington', 'Mount Washington', 'South Shore', 'South Side Flats'] 

15204 : 0  Carcinogens found
Neighborhoods in zip 15204 : ['Fairywood', 'Crafton Heights', 'Chartiers City', 'Esplen', 'Sheraden', 'Windgap', 'Elliott'] 

15205 : 0  Carcinogens found
Neighborhoods in zip 15205 : ['Oakwood', 'Westwood', 'Fairywood', 'East Carnegie', 'Crafton Heights', 'Windgap'] 

15206 : 0  Carcinogens found
Neighborhoods in zip 15206 : ['Point Breeze North', 'Squirrel Hill North', 'Garfield', 'East Liberty', 'Morningside', 'Homewood North', 'Larimer', 'Lincoln-Lemington-Belmar', 'Highland Park', 'Friendship', 'Homewood West', 'Bloomfield', 'Point Breeze', 'Stanton Heights', 'Shadyside'] 

15207 : 0  Carcinogens found
Neighborhoods in zip 15207 : ['Glen Hazel', 'Lincoln Place', 'Hays', 'Greenfield', 'New Homestead', 'Hazelwood', 'South Oakland', 'Squirrel Hill South'] 

15208 : 0  Carcin